<a href="https://www.kaggle.com/code/shapniljoy/housing-price-bangladesh?scriptVersionId=264157851" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/kaggle/input/house-price-bangladesh/house_price_bd.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import pandas as pd
import numpy as np
import re


In [ ]:
df = pd.read_csv('/kaggle/input/house-price-bangladesh/house_price_bd.csv')

In [ ]:
df.shape

There are **3865** rows (House Informations)

In [ ]:
df['Location'].str.split(',').str.get(1).nunique()

 There are **106** unique locations

In [ ]:
df['location_area'] = df['Location'].str.split(',').str.get(1).str.lower()

# Location
Replacing   
(1)The (r-a) from basundhara r-a  
(2) The ward numbers from specific wards, such as "9 no. pahartoli ward" becomes only pahartoli ward  
(3) And removing the whitespaces using str.strip()    

In [ ]:
df['location_area'] = df['location_area'].str.replace('r-a','').str.replace(r'\b\d+\s*no\.?\s*','',regex=True,flags=re.IGNORECASE).str.strip()

# Price_in_taka
(1) Removing the ৳ from prices   
(2) convert the values in Crores  
(3) rounding up the values to 2

In [ ]:
def convert_to_crores(x):
    converted = (x/10000000)
    return converted

In [ ]:
df['Price_in_Cr'] = df['Price_in_taka'].str.replace('৳','').str.replace(',','').str.strip().astype(float).apply(lambda x:convert_to_crores(x)).round(2)

In [ ]:
df.drop(['Price_in_taka'],axis=1,inplace=True)

In [ ]:
df.info()

# City
Replacing the "-city" word from narayanganj  
There are **5** cities 

In [ ]:
df['City'] = df['City'].str.replace('-city','').str.strip()

In [ ]:
df['City'].value_counts()

# Floor_no

In [ ]:
df['Floor_no'].value_counts()

In [ ]:
df['Floor_no'].str.strip()

Droping where floor_no = "Merin City - Purbach" because they seems to like outliers

In [ ]:
df = df[~df['Floor_no'].str.contains('Merin City - Purbach',na=False)]

Removing all the problematic rows with isin() function

In [ ]:
rare_floor = df['Floor_no'].value_counts()[df['Floor_no'].value_counts()<=2].index

In [ ]:
df = df[~df['Floor_no'].isin(rare_floor)]

In [ ]:
df['Floor_no'].value_counts()

# Occupancy_status
This is the most useless column , might drop it

In [ ]:
df['Occupancy_status'].value_counts()

# Bedrooms

In [ ]:
df['Bedrooms'].value_counts()

# Bathrooms

In [ ]:
df['Bathrooms'].value_counts()

In [ ]:
df.sample(3)

In [ ]:
df.to_csv('house_price_cleaned.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('house_price_cleaned.csv')

Creating new boolean features that contains specific words   
for commercial space otherwise residential space  

In [ ]:
df['is_commercial'] = df['Title'].str.contains('office|shop|commercial',case=False,na=False).astype(float)

In [ ]:
df['is_commercial'].value_counts()

In [ ]:
df['is_residential'] = df['is_commercial'] == 0

In [ ]:
df['is_residential'] = df['is_residential'].astype(float)

In [ ]:
df = df[~df['Title'].str.contains('plot|building',case=False,na=False)]

# New Column 
## price per sqft

In [ ]:
df['Price_per_sqft'] = ((df['Price_in_Cr']*10000000)/df['Floor_area']).round(2)

In [ ]:
df['Floor_no'].value_counts()

In [ ]:
df['floor_level'] = pd.cut(df['Floor_no'],bins=[0,3,8,13],labels=['low','medium','high'])

In [ ]:
df['floor_level'].value_counts()

In [ ]:
df.columns

In [ ]:
df.to_csv('feature_engineered.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
df = pd.read_csv('feature_engineered.csv')

# Univariate Analysis

## Bedrooms

(1) There are 993 missing values  
(2) Almost 75% of flat has 3 bedrooms


In [ ]:
df['Bedrooms'].isnull().sum()

In [ ]:
px.pie(df['Bedrooms'].value_counts(),values=df['Bedrooms'].value_counts(),
names=df['Bedrooms'].value_counts().index,hole=0.2,width=500,height=500).show()

## Bathrooms
(1) There are 993 missing values  
(2) Almost 60% flats has 3 bedrooms

In [ ]:
df['Bathrooms'].isnull().sum()

In [ ]:
px.pie(df['Bathrooms'].value_counts(),values=df['Bathrooms'].value_counts(),
names=df['Bathrooms'].value_counts().index,hole=0.2,width=500,height=500).show()

# floor level
(1) 684 Missing Values

In [ ]:
df['floor_level'].isnull().sum()

In [ ]:
fig = px.pie(df['floor_level'].value_counts(),values=df['floor_level'].value_counts(),
       names=df['floor_level'].value_counts().index,hole=0.0,width=600,height=600)
fig.show()

# Floor Area
(1) 99 Missing values  
(2) Huge level of outlier

In [ ]:
df['Floor_area'].isnull().sum()

In [ ]:
sns.histplot(df['Floor_area'],kde=True,bins=50)
df['Floor_area'].skew()

In [ ]:
px.box(df,x='Floor_area',width=1000,height=500)

# City

In [ ]:
df['City'].isnull().sum()

In [ ]:
px.pie(df['City'].value_counts(),values=df['City'].value_counts(),names=df['City'].value_counts().index,hole=0.2,width=600,height=600).show()

# location_area
(1) 37 missing values  
(2) Mirpur has the highest number of flats

In [ ]:
df['location_area'].isnull().sum()

In [ ]:
fig = px.pie(df['location_area'].value_counts().head(10),values=df['location_area'].value_counts().head(10),
             names=df['location_area'].value_counts().head(10).index,hole=0.2,width=600,height=600)
fig.show()

# Price in Cr
(1) Too many outliers

In [ ]:
df['Price_in_Cr'].isnull().sum()

In [ ]:
sns.histplot(df['Price_in_Cr'],kde=True,bins=50)

In [ ]:
px.box(df,x='Price_in_Cr',width=1000,height=500)

# Is commercial 
(1) Only 11% of the flats are commercial 

In [ ]:
df['is_commercial'].isnull().sum()

In [ ]:
px.pie(df['is_commercial'].value_counts(),values=df['is_commercial'].value_counts(),
names=df['is_commercial'].value_counts().index,hole=0.2,width=600,height=600).show()

# Is Residential 

In [ ]:
df['is_residential'].isnull().sum()

In [ ]:
px.pie(df['is_residential'].value_counts(),values=df['is_residential'].value_counts(),
names=df['is_residential'].value_counts().index,hole=0.2,width=600,height=600).show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('feature_engineered.csv')

# Floor Area

In [ ]:
df['Floor_area'].describe()

In [ ]:
df = df[df['Floor_area'] != 108000.000000]

In [ ]:
df = df[~df['Title'].str.contains('land',case=False,na=False)]

In [ ]:
df = df[df['Floor_area'] != 14528.0]

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
px.box(df, x='Floor_area')

In [ ]:
sns.histplot(df['Floor_area'],kde=True,bins=50)
df['Floor_area'].skew()

In [ ]:
sns.histplot(np.log1p(df['Floor_area']),kde=True,bins=50)
np.log1p(df['Floor_area']).skew()

In [ ]:
q1 = df['Floor_area'].quantile(0.25)
q3 = df['Floor_area'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr
print(lower_bound)
print(upper_bound)



In [ ]:
df[(df['Floor_area'] < lower_bound) | (df['Floor_area'] > upper_bound)].shape

In [ ]:
df[df['Floor_area'] > 3000][['location_area','Price_in_Cr']]

# Price in Cr

In [ ]:
df['Price_in_Cr'].describe()

In [ ]:
px.box(df, x='Price_in_Cr')

In [ ]:
df = df[df['Price_in_Cr'] != 0]

In [ ]:
df = df[df['Price_in_Cr'] != 0.010000]

In [ ]:
df = df[~(df['Price_in_Cr'] < 0.1)]

In [ ]:
df[df['Price_in_Cr'] > 15]

In [ ]:
df = df[df['is_commercial'] == 0]

In [ ]:
df.shape

In [ ]:
df = df[df['Price_in_Cr'] != 0.13]

In [ ]:
df = df[~(df['Price_in_Cr'] > 8)]

# Price_per_sqft

In [ ]:
sns.histplot(df['Price_per_sqft'],kde=True,bins=50)
df['Price_per_sqft'].skew()

In [ ]:
sns.histplot(np.log1p(df['Price_per_sqft']),kde=True,bins=50)
np.log1p(df['Price_per_sqft']).skew()

In [ ]:
df['Price_per_sqft'].describe()

In [ ]:
px.box(df, x='Price_per_sqft')

In [ ]:
df = df[df['Price_per_sqft'] != 30496.45]

In [ ]:
# df.loc[3177,'Floor_area'] = 4800
# df.loc[1949,'Bedrooms'] = 3
df.loc[1949,'location_area'] = 'West Sholoshohor Ward'

# df.loc[1530,'Price_per_sqft'] = (0.54*10000000)/1220

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df['Price_per_sqft'] = ((df['Price_in_Cr']*10000000)/df['Floor_area']).round(2)

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Floor_no','Occupancy_status','Location','is_commercial','is_residential'],inplace=True)

In [ ]:
df.columns

In [ ]:
df.to_csv('Outlier & partial missing.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('Outlier & partial missing.csv')

In [ ]:
df.head()

# Univariate Analysis

# (1) Bedrooms

In [ ]:
fig = px.pie(data_frame=df['Bedrooms'].value_counts(),names=df['Bedrooms'].value_counts().index,
       values=df['Bedrooms'].value_counts(),width=500,height=500,hole=0.2)

fig.update_layout(title_text='Bedrooms',title_x=0.5)
fig.show()

Bedroom is not an important column as there is not much of a variety  
78% Houses have 3 bedrooms 

# (2) Bathrooms

In [ ]:
fig = px.bar(y=df['Bathrooms'].value_counts(),width=800,height=400)

fig.update_layout(title_text='Bathrooms',title_x=0.5,xaxis_title='Bathrooms',yaxis_title='Count')
fig.show()

In [ ]:
fig = px.pie(data_frame=df['Bathrooms'].value_counts(),names=df['Bathrooms'].value_counts().index,
       values=df['Bathrooms'].value_counts(),width=500,height=500,hole=0.2)

fig.update_layout(title_text='Bathrooms',title_x=0.5)
fig.show()

63% of the Houses has 3 Bedrooms 
And the majority belongs to 3,2 & 4

# (3) Floor_area

In [ ]:
df['Floor_area'].describe()

In [ ]:
sns.histplot(df['Floor_area'],kde=True,bins=50)
print(f'Skewness: {df["Floor_area"].skew()}')
print(f'Kurtosis: {df["Floor_area"].kurt()}')


In [ ]:
sns.histplot(np.log1p(df['Floor_area']),kde=True,bins=50)
print(f'Skewness: {np.log1p(df["Floor_area"]).skew()}')
print(f'Kurtosis: {np.log1p(df["Floor_area"]).kurt()}')

In [ ]:
px.box(df,x='Floor_area',width=800,height=400)

In [ ]:
px.box(x=np.log1p(df['Price_in_Cr']),width=800,height=400)

Floor_area is right skewed and the kurtosis value is greater than 3 so even after cleaning the outliers there are some genuine extreme values. 
We can use log transformation to reduce the Skewness here  
The mean value is 1419

 # (4) City

In [ ]:
fig = px.pie(data_frame=df['City'].value_counts(),names=df['City'].value_counts().index,
       values=df['City'].value_counts(),width=500,height=500,hole=0.2)

fig.update_layout(title_text='City',title_x=0.5)
fig.show()

 Most of the Houses are Based on Dhaka and Chattogram

# (5) Location Area

In [ ]:
df['location_area'].value_counts().head(10)

In [ ]:
fig = px.bar(x=df['location_area'].value_counts().head(10).index,y=df['location_area'].value_counts().head(10),width=800,height=400)
fig.update_layout(title_text='location_area',title_x=0.5,xaxis_title='location_area',yaxis_title='Count')
fig.show()

Mirpur Has the highest Number of Houses

# (6) Price_in_Cr

In [ ]:
df['Price_in_Cr'].describe()

In [ ]:
sns.histplot(df['Price_in_Cr'],kde=True,bins=50)
print(f'Skewness: {df["Price_in_Cr"].skew()}')
print(f'Kurtosis: {df["Price_in_Cr"].kurt()}')

In [ ]:
sns.histplot(np.log1p(df['Price_in_Cr']),kde=True,bins=50)
print(f'Skewness: {np.log1p(df["Price_in_Cr"]).skew()}')
print(f'Kurtosis: {np.log1p(df["Price_in_Cr"]).kurt()}')

In [ ]:
px.box(x=df['Price_in_Cr'],width=800,height=400)

In [ ]:
px.box(x=np.log1p(df['Price_in_Cr']),width=800,height=400)


Same as the Floor Area column , we can use log transformation to redeuce the outliers and skewness  
The Average price is 0.93 Cr

# (7) Price_per_sqft 

In [ ]:
sns.histplot(df['Price_per_sqft'],kde=True,bins=50)
print(f'Skewness: {df['Price_per_sqft'].skew()}')
print(f'Kurtosis: {df['Price_per_sqft'].kurt()}')

In [ ]:
sns.histplot(np.log1p(df['Price_per_sqft']),kde=True,bins=50)
print(f'Skewness: {np.log1p(df['Price_per_sqft']).skew()}')
print(f'Kurtosis: {np.log1p(df['Price_per_sqft']).kurt()}')

In [ ]:
px.box(x=df['Price_per_sqft'],width=800,height=400)

In [ ]:
px.box(x=np.log1p(df['Price_per_sqft']),width=800,height=400)

Log transformation is needed for converging to normal distribution and outlier removal

# (8) floor_level

In [ ]:
fig = px.pie(data_frame=df['floor_level'].value_counts(),names=df['floor_level'].value_counts().index,
             values=df['floor_level'].value_counts(),width=500,height=500,hole=0.2)

fig.update_layout(title_text='floor_level',title_x=0.5)

55% of houses are on medium level 

# Feature Engineering

In [ ]:
df['Is_in_Dhaka'] = df['City'].apply(lambda x : 'yes' if x == 'dhaka' else 'no')

In [ ]:
fig = px.pie(data_frame=df['Is_in_Dhaka'].value_counts(),names=df['Is_in_Dhaka'].value_counts().index,
             values=df['Is_in_Dhaka'].value_counts(),width=500,height=500,hole=0.2)

fig.update_layout(title_text='Is_in_Dhaka',title_x=0.5)

46% houses are in dhaka 

# Bi & Multivariate Analysis

# (2) Bathrooms , Bedrooms & Price 

In [ ]:

fig = px.imshow(pd.crosstab(df['Bedrooms'],df['Bathrooms']),text_auto=True,width=500,height=500,color_continuous_scale='Hot')
fig.update_layout(title_text='Bedrooms vs Bathrooms',title_x=0.5)
fig.show()

The combination of 3 bedrooms and 3 bathrooms are the highest.  
2nd highest combination is 2 bedrooms and 2 bathrooms

In [ ]:
fig = px.imshow(pd.pivot_table(df,index='Bedrooms',columns='Bathrooms',values='Price_in_Cr',aggfunc='mean'),
          text_auto=True,width=600,height=600,color_continuous_scale='Inferno')
fig.update_layout(title_text='Bedrooms vs Bathrooms & Price',title_x=0.5)
fig.show()

The Highest Average Price is 4.24 Cr for 6 Bedrooms with 4 Bathrooms  
The Lowest Average Price is 0.2 Cr for 3 Bedrooms with 1 Bathrooms 

# (3) Bedrooms vs City

In [ ]:
fig = px.imshow(pd.crosstab(df['City'],df['Bedrooms']),text_auto=True,width=500,height=500,color_continuous_scale='Inferno')
fig.update_layout(title_text='City vs Bedrooms',title_x=0.5)
fig.show()

Chattogram and Dhaka has the highest number houses with 3 Bedrooms with equal number 690 

In [ ]:
fig = px.imshow(pd.pivot_table(df,index='City',columns='Bedrooms',values='Price_in_Cr',aggfunc='mean'),
          text_auto=True,width=600,height=600,color_continuous_scale='Inferno')
fig.update_layout(title_text='City vs Bedrooms & Price',title_x=0.5)
fig.show()

Average price of Houses with 5 and 6 Bedrooms are higher in Dhaka.  
The only house with 7 bedrooms has a lower price than 5 & 6 bedrooms , which is also in dhaka! This might be an outlier , so let's remove it 

In [ ]:
df = df[~(df['Bedrooms'] == 7)]

In [ ]:
df.columns

# (3) Floor Area

In [ ]:
sns.barplot(x=df['Bedrooms'],y=df['Floor_area'],estimator=np.median)
plt.title('Bedrooms vs Floor_area')

In [ ]:
df.drop(index=[713,186],inplace=True)

In [ ]:
sns.barplot(x=df['Bedrooms'],y=df['Floor_area'],estimator=np.median)
plt.title('Bedrooms vs Floor_area')

In [ ]:
sns.barplot(x=df['Bathrooms'],y=df['Floor_area'],estimator=np.median)
plt.title('Bathrooms vs Floor_area')

In [ ]:
sns.barplot(x=df['City'],y=df['Floor_area'],estimator=np.median)
plt.title('City vs Floor_area')

Chattogram has the highest of Median floor area available  
Narayanganj has the lowest of median floor area available

In [ ]:
sns.barplot(x=df['floor_level'],y=df['Floor_area'],estimator=np.median)
plt.title('floor_level vs Floor_area')

High Rise Houses has higher median Floor area

In [ ]:
fig = px.imshow(pd.crosstab(index=df['Bedrooms'],columns=df['location_area']),
        width=1000,height=500,color_continuous_scale='Inferno')
fig.update_layout(title_text='Bedrooms vs location_area',title_x=0.5)
fig.show()

Mirpur locality has the highest number of houses with 2 & 3 Bedrooms.

(1) Chattogram has the highest number of high rising floors with highest floor_area  
(2) Gazipur has the lowest number of low rising floors with lowest floor_area

In [ ]:
fig = px.imshow(pd.crosstab(df['Bedrooms'],df['floor_level']),text_auto=True,width=800,height=400,color_continuous_scale='Inferno')
fig.update_layout(title_text='Bedrooms vs floor_level',title_x=0.5)
fig.show()

Medium Rising floors has the highest number of 3 Bedrooms flat 

In [ ]:
fig = px.imshow(pd.pivot_table(data=df,index=df['Bedrooms'],columns=df['floor_level'],values='Price_in_Cr',aggfunc='mean'),
          text_auto=True,width=800,height=400,color_continuous_scale='Inferno')
fig.update_layout(title_text='Bedrooms vs floor_level & Price',title_x=0.5)

Highest mean price is for Medium rising 6 Bedrooms houses  
2nd highest mean price is for High rising 5 Bedrooms houses

# Target Column

In [ ]:
sns.barplot(y=df['Price_in_Cr'],x=df['Bathrooms'],estimator=np.median)
plt.title('Bathrooms vs Price')

Prices were increasing with Bathrooms but suddenly it droped at 6 bathroom , weird.

In [ ]:
df[df['Bathrooms'] == 6]

There are lot's of Houses with Bedrooms less than Bathrooms

In [ ]:
fig =px.scatter(y=df['Price_in_Cr'],x=df['Floor_area'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Floor_area vs Price',title_x=0.5,xaxis_title='Floor_area',yaxis_title='Price')
fig.show()

Prices are increasing with floor_area which is normal  
The relationship is linear

In [ ]:
fig = px.scatter(y=df['Price_in_Cr'],x=df['Floor_area'],color=df['City'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Floor_area vs Price vs City',title_x=0.5,xaxis_title='Floor_area',yaxis_title='Price')
fig.show()

Prices are higher in Dhaka in floor_area compare to other region

In [ ]:
fig = px.scatter(y=df['Price_in_Cr'],x=df['Floor_area'],color=df['floor_level'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Floor_area vs Price vs floor_level',title_x=0.5,xaxis_title='Floor_area',yaxis_title='Price')
fig.show()

In [ ]:
df[((df['floor_level'] == 'high') & (df['Floor_area'] >4000))]

The Floor_area values seems like outliers with values greater than 4000 sqft. I don't know what to do with those!  
let's keep them 

In [ ]:
df = df[~((df['Is_in_Dhaka'] == 'no') & (df['Floor_area'] > 4000))]

In chattogram bhaghmonirhat , there are 2 flats with extreme high value of Floor_area  
greater than 4500 sqft , let's remove those

In [ ]:
fig = px.scatter(y=df['Price_in_Cr'],x=df['Floor_area'],color=df['Is_in_Dhaka'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Floor_area vs Price vs Is_in_Dhaka',title_x=0.5,xaxis_title='Floor_area',yaxis_title='Price')
fig.show()

In [ ]:
fig =px.scatter(y=df['Price_in_Cr'],x=df['Price_per_sqft'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Price_per_sqft vs Price',title_x=0.5,xaxis_title='Price_per_sqft',yaxis_title='Price')
fig.show()

The relationship between Price and Price per sqft is linear which is normal to assume

In [ ]:
df = df[~((df['City'].isin(['cumilla'])) & (df['Price_per_sqft'] > 6000))]

Found an outlier value for cumilla with price per sqft is at 10k , which is a business space. Removing that row

In [ ]:
df = df[~((df['City'].isin(['gazipur'])) & (df['Price_per_sqft'] > 6000))]

Found an outlier value for gazipur with price per sqft is at 12.5k , which is a warehouse. Removing that row

In [ ]:
fig =px.scatter(y=df['Price_in_Cr'],x=df['Price_per_sqft'],color=df['City'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text='Price_per_sqft vs Price vs City',title_x=0.5,xaxis_title='Price_per_sqft',yaxis_title='Price')
fig.show()

In Dhaka , the spread between price and price per sqft is higher  
The spread in chattogram is more stable compare to other region  
cumilla and gazipur have a non linear trend

In [ ]:
fig = px.scatter(y=df['Price_in_Cr'],x=df['Price_per_sqft'],color=df['floor_level'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text = 'Price_per_sqft vs Price vs floor_level',title_x=0.5,xaxis_title='Price_per_sqft',yaxis_title='Price')
fig.show()

The price vs Price per sqft relation is non-linear for High Rising floors

In [ ]:
df[((df['floor_level'].isin(['medium'])) & (df['Price_per_sqft'] > 24000))]

In [ ]:
df[df['location_area'] == 'banani dohs']

In [ ]:
fig = px.scatter(y=df['Price_in_Cr'],x=df['Price_per_sqft'],color=df['Is_in_Dhaka'],width=800,height=400,trendline='lowess')
fig.update_layout(title_text = 'Price_per_sqft vs Price vs Is_in_Dhaka',title_x=0.5,xaxis_title='Price_per_sqft',yaxis_title='Price')
fig.show()

In [ ]:
sns.barplot(data=df,y='Price_in_Cr',x='City',estimator=np.median)

Dhaka and Chattogram has simillar median price in Cr around 80 lacs  
Gazipur has the lowest median price in Cr around 40 lacs 

In [ ]:
sns.barplot(data=df,y='Price_in_Cr',x='Is_in_Dhaka',estimator=np.median)

Dhaka has higher median price than other region combined



### 🏠 **General Observations**
- **Bedrooms**: Not a strong differentiator — **78%** of houses have **3 bedrooms**, and **63%** have **3 bathrooms**. Most combinations are **3 beds & 3 baths**, followed by **2 beds & 2 baths**.
- **Floors**: Majority of houses are on **2, 3, or 4 floors**.
- **Regions**: Most houses are located in **Dhaka** and **Chattogram**, with **Mirpur** having the highest concentration.



### 📊 **Numerical Features**
#### **Floor Area**
- **Right-skewed** distribution with **kurtosis > 3**, indicating presence of genuine extreme values.
- **Mean floor area**: ~1419 sqft.
- **Log transformation** recommended to reduce skewness and manage outliers.
- **Outliers**:
  - Values > 4000 sqft are potential outliers but retained.
  - **Bhaghmonirhat (Chattogram)** has 2 extreme values > 4500 sqft — removed.

#### **Price**
- **Average price**: ~0.93 Cr.
- **Log transformation** suggested for normalization and outlier handling.
- **Highest average price**: 4.24 Cr for **6 beds & 4 baths**.
- **Lowest average price**: 0.2 Cr for **3 beds & 1 bath**.
- **Price increases with floor area** — **linear relationship**.

#### **Price per Sqft**
- **Highly correlated** with target price.
- **Outliers**:
  - **Cumilla**: 10k/sqft (business space) — removed.
  - **Gazipur**: 12.5k/sqft (warehouse) — removed.
- **Linear relationship** with price overall, but **non-linear** for **high-rise floors**.



### 🏢 **Location-Based Insights**
- **Dhaka & Chattogram**: Highest number of houses with 3 bedrooms (690 each).
- **Dhaka**: Higher median price than other regions combined.
- **Gazipur**: Lowest median price (~40 lacs).
- **Chattogram**: More stable price-per-sqft spread.
- **Cumilla & Gazipur**: Show **non-linear** trends in price vs. price-per-sqft.


### 🏗️ **Building Type Insights**
- **High-rise houses**: Higher median floor area.
- **Medium-rise houses**:
  - Most common for 3-bedroom flats.
  - Highest mean price for 6-bedroom units.
- **Low-rise houses**: Least floor area, especially in Gazipur.


### 🛁 **Bathroom Trends**
- Price generally increases with number of bathrooms, but drops unexpectedly at **6 bathrooms** — possibly an anomaly.
- Some houses have **more bathrooms than bedrooms**, which is unusual.



# Missing_Value_Imputation

In [ ]:
df['Bedrooms'].fillna(df['Bedrooms'].median(),inplace=True)

In [ ]:
df['Bathrooms'].fillna(df['Bathrooms'].median(),inplace=True) 

In [ ]:
df['floor_level'].fillna(df['floor_level'].mode()[0],inplace=True)


In [ ]:
df['Floor_area'].fillna(df['Floor_area'].median(),inplace=True)

In [ ]:
df['Price_per_sqft'].fillna(df['Price_per_sqft'].median(),inplace=True)

In [ ]:
df['Price_in_Cr'].isnull().sum()

In [ ]:
df.to_csv('final_data.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('final_data.csv')

In [ ]:
df = df.drop(['Title'],axis=1)

# Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder,TargetEncoder,OrdinalEncoder
df[['City','floor_level','Is_in_Dhaka']] = OrdinalEncoder(handle_unknown='error').fit_transform(df[['City','floor_level','Is_in_Dhaka']])

In [ ]:
df[['location_area']] = TargetEncoder().fit_transform(df[['location_area']],df[['Price_in_Cr']])

In [ ]:
x = df.drop(['Price_in_Cr'],axis=1)
y = df['Price_in_Cr']

# Scalling

In [ ]:
from sklearn.preprocessing import StandardScaler

x_scaled = StandardScaler().fit_transform(x)


# (1) Correlation

In [ ]:
px.imshow(df.corr(),text_auto=True,width=800,height=600)

In [ ]:
fi_df1 = df.corr()['Price_in_Cr'].to_frame().sort_values(by='Price_in_Cr',ascending=False).reset_index().rename({'index':'Features','Price_in_Cr':'Importance'},axis=1)

In [ ]:
fi_df1

In [ ]:
fi_df1.tail(3)

# (2) Random Forest Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100,random_state=0).fit(x_scaled,y)

fi_df2 = pd.DataFrame({'Features':x.columns,'RF_Importance':rf.feature_importances_}).sort_values(by='RF_Importance',ascending=False)
fi_df2

In [ ]:
fi_df2.tail(3)

# (3) Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100,random_state=0).fit(x_scaled,y)

fi_df3 = pd.DataFrame({'Features':x.columns,'GB_Importance':gb.feature_importances_}).sort_values(by='GB_Importance',ascending=False)
fi_df3

# (4) Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1).fit(x_scaled,y)

fi_df4 = pd.DataFrame({'Features':x.columns,'lasso_Importance': lasso.coef_}).sort_values(by='lasso_Importance',ascending=False)
fi_df4

# (5) Recursive Feature Elimination
## Backward Subset Selection (mlxtend) 

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(estimator=RandomForestRegressor(n_estimators=100,random_state=0)
          ,n_features_to_select=x.shape[0]).fit(x_scaled,y)

features = x.columns[rfe.support_]
Importance = rfe.estimator_.feature_importances_

fi_df5 = pd.DataFrame({'Features':features,'RFE_Importance':Importance}).sort_values(by='RFE_Importance',ascending=False)
fi_df5

# (6) Permutation Importance

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size=0.2,random_state=0)

rf = RandomForestRegressor(n_estimators=100,random_state=0)
rf.fit(x_train,y_train)

pi = permutation_importance(estimator=rf,X=x_test,y=y_test,scoring='neg_mean_squared_error',n_repeats=10,random_state=0)

fi_df6 = pd.DataFrame({'Features':x.columns,'PI_Importance':pi.importances_mean}).sort_values(by='PI_Importance',ascending=False)
fi_df6

# (7) Shap values

In [ ]:
import shap
explainer = shap.TreeExplainer(rf)
shap_values = explainer(x_test,check_additivity=False)

shap.summary_plot(shap_values,x_test,feature_names=x.columns)

In [ ]:
fi_df7 = pd.DataFrame({'Features':x.columns,'Shap_Importance':np.abs((shap_values.values).mean(axis=0))}).sort_values(by='Shap_Importance',ascending=False)
fi_df7

# Merge ALL fi_df

In [ ]:
final_df = fi_df1.merge(fi_df2,on='Features').merge(fi_df3,on='Features').merge(fi_df4,on='Features').merge(fi_df5,on='Features').merge(fi_df6,on='Features').merge(fi_df7,on='Features').set_index('Features')

In [ ]:
final_df

## Normalization (Dividing each column by it's Column sum)

In [ ]:
final_df.divide(final_df.sum(axis=0),axis=1)

In [ ]:
final_df.mean(axis=1).sort_values(ascending=False).to_frame('Importance')

In [ ]:
from sklearn.model_selection import cross_val_score,KFold

cv_original = cross_val_score(estimator=rf,X=x_scaled,y=y,scoring='neg_mean_squared_error',cv=10)
cv_original.mean()

In [ ]:
cv_after = cross_val_score(estimator=rf,X=x.drop(['Is_in_Dhaka','City','floor_level','Bedrooms','Bathrooms'],axis=1),
                           y=y,scoring='neg_mean_squared_error',cv=10)
cv_after.mean()

Main Features are Price_per_sqft , Floor_area and location_area,  
After removing other columns the cross validation MSE reached lowest at 0.0098

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

Removing the Price_per_sqft column because customers shouldn't have a knowledge about the Price_per_sqft either

In [ ]:
x = df.drop(['Price_in_Cr'],axis=1)
y = df['Price_in_Cr']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PowerTransformer,OneHotEncoder,OrdinalEncoder
from category_encoders import TargetEncoder  
from sklearn.linear_model import LinearRegression

In [ ]:
transformer = ColumnTransformer([
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
    # ('Target_Encoding',TargetEncoder(),['location_area']),
    ('Scaler',StandardScaler(),['Floor_area'])
],remainder='passthrough')

model_pipe = Pipeline([
    ('preprocessing',transformer),
    ('model',LinearRegression())
])



In [ ]:
from sklearn.model_selection import cross_val_score,KFold

cv_original = cross_val_score(estimator=model_pipe,X=x_train,y=y_train,scoring='r2',cv=KFold(n_splits=10,shuffle=True,random_state=0))
cv_original.mean()

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

model_pipe.fit(x_train,y_train)
y_pred = model_pipe.predict(x_test)

MSE = mean_squared_error(y_test,y_pred)
MAE = mean_absolute_error(y_test,y_pred)

print(f'Mean Squared Error: {MSE}')

print(f'Mean Absolute Error: {MAE}')


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'lasso': Lasso(), 
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': XGBRegressor(),
    'lightgbm':LGBMRegressor()
}

# (1) Without Feature Transformation

In [ ]:
transformer = ColumnTransformer([
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
    # ('Target_Encoding',TargetEncoder(),['location_area']),
    ('Scaler',StandardScaler(),['Floor_area'])
],remainder='passthrough')


def score(model_name,model):

    output = []
    output.append(model_name)

    model_pipe = Pipeline([
        ('preprocessor',transformer),
        ('regressor',model)
    ])

    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

    # cv = cross_val_score(model_pipe,x_train,y_train,cv=KFold(n_splits=10,shuffle=True,random_state=0),scoring='r2') 
    # output.append(cv.mean())

    model_pipe.fit(x_train,y_train)
    y_pred = model_pipe.predict(x_test)

    R2 = r2_score(y_test,y_pred)
    MSE = mean_squared_error(y_test,y_pred)
    MAE = mean_absolute_error(y_test,y_pred)

    output.append(R2)
    output.append(MSE)
    output.append(MAE)

    return output


model_output  = []
for model_name,model in model_dict.items():
    model_output.append(score(model_name,model))

model_output = pd.DataFrame(model_output,columns=['model_name','r2_score','mse','mae']).sort_values(by='mse',ascending=True)
model_output

# (2) With Feature Transformation (Linear Assumptions)

In [ ]:
x_train2,x_test2,y_train2,y_test2 = train_test_split(x,y,test_size=0.2,random_state=0)


floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
    # ('Target_Encoding',TargetEncoder(random_state=0),['location_area']),
    # ('Scaler',StandardScaler(),['Floor_area'])
],remainder='passthrough')


def score_transformed(model_name,model):

    output = []
    output.append(model_name)

    model_pipe = Pipeline([
        ('preprocessor',transformer),
        ('regressor',model)
    ])

    final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)


    # cv = cross_val_score(final_pipe,x_train2,y_train2,cv=KFold(n_splits=10,shuffle=True,random_state=0),scoring='r2') 
    # output.append(cv.mean())

    final_pipe.fit(x_train2,y_train2)
    y_pred2 = final_pipe.predict(x_test2)

    R2 = r2_score(y_test2,y_pred2)
    MSE = mean_squared_error(y_test2,y_pred2)
    MAE = mean_absolute_error(y_test2,y_pred2)

    output.append(R2)
    output.append(MSE)
    output.append(MAE)

    return output



model_output_transformed  = []
for model_name,model in model_dict.items():
    model_output_transformed.append(score_transformed(model_name,model))

model_output_transformed = pd.DataFrame(model_output_transformed,
                                        columns=['model_name','r2_score','mse','mae']).sort_values(by='mse',ascending=True)
model_output_transformed

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,TargetEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import optuna
from plotly.io import show
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

In [ ]:
x = df.drop('Price_in_Cr',axis=1)
y = df['Price_in_Cr']

# Optuna

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)


floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
],remainder='passthrough')


def objective(trial,preprocessor):
    regressor_name = trial.suggest_categorical('regressor',['GradientBoostingRegressor'])

    if regressor_name == 'GradientBoostingRegressor':
        n_estimators = trial.suggest_int('model__n_estimators',50,300)
        learning_rate = trial.suggest_float('model__learning_rate',0.05,0.3)
        max_features = trial.suggest_categorical('model__max_features',['sqrt', 'log2'])
        subsample = trial.suggest_float('model__subsample',0.7,1.0)
        max_depth = trial.suggest_int('model__max_depth',3,6)
        min_samples_leaf = trial.suggest_int('model__min_samples_leaf',1,5)
        min_samples_split = trial.suggest_int('model__min_samples_split',2,5)


    model_pipe = Pipeline([
        ('preprocessor',transformer),
        ('model',GradientBoostingRegressor(n_estimators=n_estimators,learning_rate=learning_rate,max_features=max_features,
                                           max_depth=max_depth,min_samples_leaf=min_samples_leaf,
                                           min_samples_split=min_samples_split,subsample=subsample,random_state=0))
                                           ])

    final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

    trial.set_user_attr('model',final_pipe)

    cv_score = cross_val_score(final_pipe,x_train,y_train,cv=KFold(n_splits=10,shuffle=True,random_state=0),scoring='neg_mean_squared_error')
    return cv_score.mean()


study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=0),pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial : objective(trial,preprocessor=transformer),n_trials=100,n_jobs=-1)
print(f'Best neg score: {study.best_value}')
print(f'Best Params: {study.best_params}')

final_model = study.best_trial.user_attrs['model']
final_model.fit(x_train,y_train)
y_pred = final_model.predict(x_test)

print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'R2: {r2_score(y_test,y_pred)}')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,TargetEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import optuna
from plotly.io import show
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

In [ ]:
x = df.drop('Price_in_Cr',axis=1)
y = df['Price_in_Cr']

# For Tree Dump

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

transformer = ColumnTransformer([
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
    # ('Target_Encoding',TargetEncoder(),['location_area']),
    ('Scaler',StandardScaler(),['Floor_area'])
],remainder='passthrough')

model_pipe = Pipeline([
    ('preprocessing',transformer),
    ('model',XGBRegressor(random_state=0,n_jobs=1,n_estimators=1,gamma=10,min_child_weight=10))
])


# final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

model = model_pipe.fit(x_train,y_train)
y_pred = model.predict(x_test)

MSE = mean_squared_error(y_test,y_pred)
MAE= mean_absolute_error(y_test,y_pred)
R2 = r2_score(y_test,y_pred)

print(f'Base XGboost Model\nMSE:{MSE}\nMAE:{MAE}\nR2:{R2}')


In [ ]:
tree_dump = model.steps[1][1].get_booster().get_dump(dump_format='text',with_stats=True)

for trees in tree_dump:
    print(trees)

# Optuna

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
],remainder='passthrough')


def objective(trial,preprocessor):
     regressor_name = trial.suggest_categorical('regressor',['XGBRegressor'])

     if regressor_name == 'XGBRegressor':
        
        gamma = trial.suggest_int('model__gamma',0,0.5)
        reg_lambda = trial.suggest_float('model__reg_lambda',1,2)
        n_estimators = trial.suggest_int('model__n_estimators',50,300)
        learning_rate = trial.suggest_float('model__eta',0.05,0.3)
        min_child_weight = trial.suggest_int('model__min_child_weight',1,3)
        subsample = trial.suggest_float('model__subsample',0.7,1)
        colsample_bytree = trial.suggest_float('model__colsample_bytree',0.7,1)
        max_depth = trial.suggest_int('model__max_depth',3,6)
        tree_method = trial.suggest_categorical('model__tree_method',['hist'])
        max_bin = trial.suggest_int('model__max_bin',64,256)

        model_pipe = Pipeline([
        ('preprocessor',transformer),
        ('model',XGBRegressor(random_state=0,gamma=gamma,reg_lambda=reg_lambda,
                              n_estimators=n_estimators,learning_rate=learning_rate,min_child_weight=min_child_weight,
                              subsample=subsample,colsample_bytree=colsample_bytree,
                              max_depth=max_depth,tree_method=tree_method,max_bin=max_bin))
                              ])
        
        final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

        trial.set_user_attr('model',final_pipe)

        cv = cross_val_score(final_pipe,x_train,y_train,cv=KFold(n_splits=5,shuffle=True,random_state=0),scoring='neg_mean_squared_error')

        return cv.mean()
     

study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=0),pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial: objective(trial,transformer),n_trials=100,n_jobs=-1)

print(f'Best neg score: {study.best_value}')
print(f'Best Params: {study.best_params}')

final_model = study.best_trial.user_attrs['model']
final_model.fit(x_train,y_train)
y_pred = final_model.predict(x_test)

print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'R2: {r2_score(y_test,y_pred)}')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,TargetEncoder,PowerTransformer
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import optuna
from plotly.io import show
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df = pd.read_csv('final_data.csv')
df = df.drop(['Title','Price_per_sqft','Is_in_Dhaka'],axis=1)
df.head()

In [ ]:
x = df.drop('Price_in_Cr',axis=1)
y = df['Price_in_Cr']

# Optuna 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

floor_area_pipe = Pipeline([
    ('log_transform',PowerTransformer(method='yeo-johnson')),
    ('scalar',StandardScaler())
])

transformer = ColumnTransformer([
    ("floor_area",floor_area_pipe,['Floor_area']),
    ("log_transform",PowerTransformer(method='yeo-johnson'),['Floor_area']),
    ('Onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),['City','Bedrooms','Bathrooms','location_area']),
    ('Ordinal',OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1),['floor_level']),
],remainder='passthrough')


def objective(trial,preprocessor):
     regressor_name = trial.suggest_categorical('regressor',['SVR'])

     if regressor_name == 'SVR':
        kernel = trial.suggest_categorical('model__kernel',['rbf','linear','poly','sigmoid'])
        C = trial.suggest_float('model__C',0.01,1000)
        epsilon = trial.suggest_float('model__epsilon', 0.01,1)
        gamma = trial.suggest_categorical('model__gamma', ['scale', 'auto'])

        if kernel == 'poly':
            degree = trial.suggest_int('model__degree',2,5)
            coef0 = trial.suggest_float('model__coef0',0.01,1)
        elif kernel == 'sigmoid':
            coef0 = trial.suggest_float('model__coef0',0.01,1)
        else:
            degree = 3
            coef0 = 0.0


        model_pipe = Pipeline([
            ('preprocessing',transformer),
            ('model',SVR(kernel=kernel,C=C,epsilon=epsilon,gamma=gamma,degree=degree if kernel == 'poly' else 3,coef0=coef0))
        ])

                              
        final_pipe = TransformedTargetRegressor(regressor=model_pipe,func=np.log1p,inverse_func=np.expm1)

        trial.set_user_attr('model',final_pipe)

        cv = cross_val_score(final_pipe,x_train,y_train,cv=KFold(n_splits=5,shuffle=True,random_state=0),scoring='neg_mean_squared_error')

        return cv.mean()
     

study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=0),pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial: objective(trial,transformer),n_trials=100,n_jobs=-1)

print(f'Best neg score: {study.best_value}')
print(f'Best Params: {study.best_params}')

final_model = study.best_trial.user_attrs['model']
final_model.fit(x_train,y_train)
y_pred = final_model.predict(x_test)

print(f'MSE: {mean_squared_error(y_test,y_pred)}')
print(f'MAE: {mean_absolute_error(y_test,y_pred)}')
print(f'R2: {r2_score(y_test,y_pred)}')

In [ ]:
final_model

In [ ]:
import pickle

with open('SVR_model.pkl','wb') as f:
    pickle.dump(final_model,f)